# Ered_SKy

In [1]:
import pickle 
import numpy as np
import pandas as pd
import datetime as dt
import time
import os
pd.set_option('display.max_rows',6000)
pd.set_option('display.max_columns',1000)

In [2]:
def SkySports_Date(sd):
    Sky_Months_Date={'January':'01','February':'02','March':'03','April':'04','May':'05',
            'June':'06','July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'}
    if type(sd)==pd._libs.tslibs.timestamps.Timestamp or type(sd)==dt.datetime:
        return sd,'NOTIME'
    else:
        sd=sd.split(',')
        time=sd[0]
        date=sd[1]
        #time
        if time[-2:]=='pm':
            time=time[:-2]
            time=time.split(':')
            if int(time[0])==12:
                H=(int(time[0])*100)
            else:
                H=(int(time[0])*100)+1200
            M=int(time[1])
            timez=H+M
        elif time[-2:]=='am':
            time=time[:-2]
            time=time.split(':')
            H=(int(time[0])*100)
            M=int(time[1])
            timez=H+M
        else:
            print('WTF, SkySports_Date-Time')
            return 'Erro'

        #date
        date=date.split(' ')
        D=date[2][:-2]
        M=Sky_Months_Date[date[3]]
        Y=date[4][:-1]
        data=D+'/'+M+'/'+Y
        dataz=dt.datetime.strptime(data,"%d/%m/%Y")

        return dataz, timez

In [232]:
def Prepare__Df(df,season,league): 
    df = df.dropna(axis=0,how='all') #retira as linhas cujos valores são todos NaN

    print('Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"')
    tydate = np.array(list(map(lambda x:type(x),list(df['Date']))))
    tstamp = list(map(lambda x:type(x)==pd._libs.tslibs.timestamps.Timestamp,list(df['Date'])))
    if sum(tstamp)==len(df):
        stamp = True
    
    elif len(np.where(tydate==str)[0])==len(df):
        ok = np.array(list(map(lambda x:(x[4]=='-') and (x[7]=='-'),list(df['Date']))))
        if not (ok.all()):
            print('Man, as datas não estão bem')
            ok2 = list(np.where(list(ok)==False)[0])
            print(str(ok2),'São os índices maus' )
            raise('aa')
        stamp = False
    else:
        print(list(map(lambda x:type(x),list(df['Date']))))
        raise Exception('OOps, temos vários tipos de datas nesta df')
    # A parte em cima serve só para ver as datas e verificar que podemos fazer o Prepare__Df
    if not stamp: #vamos passar para datetime (o timestamp é basicamente o mesmo do datetime)
        df['Date']=list(map(lambda x:dt.datetime.strptime(x,'%Y-%m-%d'),list(df['Date'])))
    if season==2020: #Corona season
        df1=df.loc[list(np.where(np.logical_and(np.logical_and(df['Date']>=dt.datetime(2019,8,4),df['Date']<=dt.datetime(2020,8,3)),df['League']==league))[0])]
    elif season==2021: #After Corona season
        df1=df.loc[list(np.where(np.logical_and(np.logical_and(df['Date']>=dt.datetime(2020,8,8),df['Date']<=dt.datetime(2021,8,3)),df['League']==league))[0])]    
    else:
        df1=df.loc[list(np.where(np.logical_and(np.logical_and(df['Date']>=dt.datetime((season-1),8,1),df['Date']<=dt.datetime(season,8,1)),df['League']==league))[0])]
        print('Lembrar de verificar as datas limite')  
    
    try:  
        return df1.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    except: #Under não tem Time
        return df1.sort_values(['Date'],ascending=[True]).reset_index(drop=True)

In [240]:
skys = os.listdir('C://Users//joaom//Desktop//WScrapy2//Sky')
Sky = [i for i in skys if 'csv' in i and ('games' not in i)]

In [241]:
Sky

['Sky230_260.csv',
 'Sky260_290.csv',
 'Sky290_310.csv',
 'Sky310_336GF.csv',
 'Sky336_340.csv',
 'Sky340_365.csv',
 'Sky365_385.csv',
 'Sky385_400.csv',
 'Sky400_425.csv']

In [447]:
Ered=[]
for df in Sky:
    print('C://Users//joaom//Desktop//WScrapy2//Sky//'+df)
    df1 = pd.read_csv('C://Users//joaom//Desktop//WScrapy2//Sky//'+df)
    df1 = df1.loc[df1['League']==' Scottish Premiership']
    Ered+=[df1]

C://Users//joaom//Desktop//WScrapy2//Sky//Sky230_260.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky260_290.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky290_310.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky310_336GF.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky336_340.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky340_365.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky365_385.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky385_400.csv
C://Users//joaom//Desktop//WScrapy2//Sky//Sky400_425.csv


In [448]:
Ered1 = pd.concat(Ered,sort=False)

In [449]:
len(Ered1)

2233

In [450]:
dtime = list(map(lambda x:SkySports_Date(x),list(Ered1['Date'])))
time = [i[1] for i in dtime]
date = [i[0] for i in dtime]
Ered1['Date'] = date
Ered1['Time'] = time

In [451]:
Ered1=Ered1.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [452]:
Ered1.iloc[0]['Date']

Timestamp('2011-01-02 00:00:00')

In [25]:
Ered1['Date']>=dt.datetime((season-1),8,1)

632     False
652     False
247     False
653     False
2124    False
        ...  
8509     True
8511     True
8512     True
8513     True
8510     True
Name: Date, Length: 2519, dtype: bool

In [453]:
Sered=[]
for season in range(2012,2021,1):
    if season==2020: #Corona season
        ok=Ered1.loc[np.array(np.where(np.logical_and(Ered1['Date']>=dt.datetime(2019,8,4),Ered1['Date']<=dt.datetime(2020,8,3)))[0])] 
    else:
        ok=Ered1.loc[np.array(np.where(np.logical_and(Ered1['Date']>=dt.datetime((season-1),8,1),Ered1['Date']<=dt.datetime(season,8,1)))[0])]
        #print('Lembrar de verificar as datas limite') 
    print(season,len(ok))
    Sered+=[ok.reset_index(drop=True)]

2012 256
2013 318
2014 242
2015 232
2016 228
2017 228
2018 219
2019 228
2020 194


In [454]:
6*38

228

In [465]:
for i in range(len(Sered)):
    vars()['Ered_20'+str(12+i)]=Sered[i].sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [466]:
for i in range(len(Sered)):
    vars()['Teams_20'+str(12+i)]=list(np.unique(Sered[i]['HT']))

In [57]:
Ered_2012=Sered[0]

In [62]:
Teams_2012=list(np.unique(Ered_2012['HT']))

In [265]:
len(Teams_2012)

24

In [457]:
def teams(T,df,season,mode=1):
    print(season,'--------------------------------')
    if mode==1: 
        T_copy=T.copy()
        games=[]
        for t in T:
            for t2 in T:
                if t2==t:
                    continue
                games+=[str(t)+'_'+str(t2),str(t2)+'_'+str(t)]
        for i in range(len(df)):
            G = str(df.iloc[i]['HT'])+'_'+str(df.iloc[i]['AT'])
            if G not in games:
                print('Game Not IN')
                print(df.iloc[i]['HT'],' Vs ',df.iloc[i]['AT'])
    elif mode==2:
        T_copy=T.copy()
        games={}
        for t in T:
            for t2 in T:
                if t2==t:
                    continue
                games[str(t)+'_'+str(t2)]=[]
                games[str(t2)+'_'+str(t)]=[]
        for i in range(len(df)):
            G = str(df.iloc[i]['HT'])+'_'+str(df.iloc[i]['AT'])
            if G in games:
                games[G]+=[i]
                if len(games[G])>1:
                    print('Double Game')
                    print(df.iloc[i]['HT'],' Vs ',df.iloc[i]['AT'])
        return games

In [67]:
teams(Teams_2012,Ered_2012)

In [467]:
for s in range(2012,2021,1):
    vars()['G_'+str(s)]=teams(vars()['Teams_'+str(s)],vars()['Ered_'+str(s)],s,mode=2)

2012 --------------------------------
Double Game
Aberdeen  Vs  Celtic
Double Game
Kilmarnock  Vs  Hibernian
Double Game
Motherwell  Vs  Rangers
Double Game
Kilmarnock  Vs  Hearts
Double Game
Rangers  Vs  Aberdeen
Double Game
Hearts  Vs  Hibernian
Double Game
Rangers  Vs  Celtic
Double Game
St. Johnstone  Vs  Hearts
Double Game
Rangers  Vs  Kilmarnock
Double Game
Hibernian  Vs  St. Johnstone
Double Game
Aberdeen  Vs  Dunfermline Athletic
Double Game
Hearts  Vs  Celtic
Double Game
Hearts  Vs  Rangers
Double Game
Celtic  Vs  Aberdeen
Double Game
Motherwell  Vs  Celtic
Double Game
St. Johnstone  Vs  Aberdeen
Double Game
Kilmarnock  Vs  Rangers
Double Game
Motherwell  Vs  Hibernian
Double Game
Dundee United  Vs  Celtic
Double Game
St. Johnstone  Vs  Aberdeen
Double Game
St. Johnstone  Vs  Celtic
Double Game
Aberdeen  Vs  Kilmarnock
Double Game
Celtic  Vs  Dundee United
Double Game
Dunfermline Athletic  Vs  Hibernian
Double Game
Hearts  Vs  St Mirren
Double Game
St Mirren  Vs  Celtic
Double

Aberdeen  Vs  Partick Thistle
Double Game
St Mirren  Vs  Ross County
Double Game
Aberdeen  Vs  Inverness CT
Double Game
Celtic  Vs  Motherwell
Double Game
Hibernian  Vs  St Mirren
Double Game
Partick Thistle  Vs  Kilmarnock
Double Game
Ross County  Vs  Dundee United
Double Game
St. Johnstone  Vs  Hearts
Double Game
Kilmarnock  Vs  Inverness CT
Double Game
Motherwell  Vs  Aberdeen
Double Game
Ross County  Vs  Hearts
Double Game
St Mirren  Vs  Partick Thistle
Double Game
Hibernian  Vs  Celtic
Double Game
Hearts  Vs  St Mirren
Double Game
Kilmarnock  Vs  Ross County
Double Game
Partick Thistle  Vs  Dundee United
Double Game
St Mirren  Vs  Aberdeen
Double Game
Dundee United  Vs  Kilmarnock
Double Game
Hibernian  Vs  Ross County
Double Game
Inverness CT  Vs  Hearts
Double Game
Motherwell  Vs  Partick Thistle
Double Game
Celtic  Vs  St. Johnstone
Double Game
Dundee United  Vs  Motherwell
Double Game
Hearts  Vs  Celtic
Double Game
Kilmarnock  Vs  Hibernian
Double Game
Partick Thistle  Vs  Abe

Dundee  Vs  Hamilton Academical
Double Game
Hearts  Vs  Ross County
Double Game
Kilmarnock  Vs  Partick Thistle
Double Game
Motherwell  Vs  St. Johnstone
Double Game
Celtic  Vs  Aberdeen
Double Game
Dundee United  Vs  Partick Thistle
Double Game
St. Johnstone  Vs  Celtic
Double Game
Dundee  Vs  Kilmarnock
Double Game
Hamilton Academical  Vs  Inverness CT
Double Game
Ross County  Vs  Motherwell
Double Game
Aberdeen  Vs  Hearts
Double Game
Inverness CT  Vs  Dundee
Double Game
Kilmarnock  Vs  Dundee United
Double Game
Partick Thistle  Vs  Hamilton Academical
Double Game
Aberdeen  Vs  Ross County
Double Game
Celtic  Vs  Motherwell
Double Game
Hearts  Vs  St. Johnstone
2017 --------------------------------
Double Game
Hamilton Academical  Vs  Inverness CT
Double Game
Ross County  Vs  Motherwell
Double Game
Celtic  Vs  Aberdeen
Double Game
Hearts  Vs  Rangers
Double Game
Partick Thistle  Vs  St. Johnstone
Double Game
Motherwell  Vs  Hearts
Double Game
Aberdeen  Vs  Partick Thistle
Double Gam

Double Game
Celtic  Vs  Ross County
Double Game
Hamilton Academical  Vs  Livingston
Double Game
Motherwell  Vs  Hibernian
Double Game
St. Johnstone  Vs  Kilmarnock
Double Game
St Mirren  Vs  Aberdeen
Double Game
Hearts  Vs  Rangers
Double Game
Kilmarnock  Vs  Ross County
Double Game
Rangers  Vs  Aberdeen
Double Game
St. Johnstone  Vs  Hearts
Double Game
Hamilton Academical  Vs  Celtic
Double Game
Aberdeen  Vs  St. Johnstone
Double Game
Hearts  Vs  Kilmarnock
Double Game
Motherwell  Vs  Celtic
Double Game
Rangers  Vs  Hibernian
Double Game
Ross County  Vs  Livingston
Double Game
St Mirren  Vs  Hamilton Academical
Double Game
Hamilton Academical  Vs  Aberdeen
Double Game
Celtic  Vs  Hearts
Double Game
Hibernian  Vs  Ross County
Double Game
Kilmarnock  Vs  Rangers
Double Game
Livingston  Vs  St Mirren
Double Game
St. Johnstone  Vs  Motherwell
Double Game
Hearts  Vs  Hamilton Academical
Double Game
Ross County  Vs  St. Johnstone
Double Game
Aberdeen  Vs  Celtic
Double Game
Rangers  Vs  Liv

In [270]:
DG_2014

[[272, 288],
 [75, 144],
 [46, 73],
 [281, 286],
 [307, 312],
 [137, 143],
 [267, 276],
 [240, 247],
 [298, 324],
 [261, 266],
 [268, 292, 293],
 [303, 323],
 [275, 289],
 [189, 218],
 [138, 142],
 [116, 120],
 [277, 291],
 [109, 145],
 [299, 317],
 [273, 290],
 [241, 246],
 [125, 131],
 [302, 325],
 [85, 92],
 [269, 274],
 [88, 91],
 [115, 121],
 [47, 53]]

In [468]:
for s in range(2012,2021,1):
    vars()['DG_'+str(s)]=[]
    for key in vars()['G_'+str(s)]:
        if len(vars()['G_'+str(s)][key])>1:
            vars()['DG_'+str(s)]+=[vars()['G_'+str(s)][key]]

In [469]:
for s in range(2012,2021,1):
    print(s,'-----------------------')
    vars()['Dfs_'+str(s)]=[]
    for i in range(len(vars()['DG_'+str(s)])):
        ok1 = vars()['Ered_'+str(s)].loc[np.array(vars()['DG_'+str(s)][i])]
        vars()['Dfs_'+str(s)]+=[ok1]

2012 -----------------------
2013 -----------------------
2014 -----------------------
2015 -----------------------
2016 -----------------------
2017 -----------------------
2018 -----------------------
2019 -----------------------
2020 -----------------------


In [154]:
list(Dfs_2012[2].sort_values('GameID',ascending=False).index[1:])
#Dfs_2012[2]
#for s in range(2012,2021,1):
#    for i in range(len(vars()['Dfs_'+str(s)])):
#        if len(vars()['Dfs_'+str(s)][i])>2:
#            print(s,i)
    

[126, 117]

In [470]:
for s in range(2012,2021,1):
    vars()['Drop_'+str(s)]=[]
    for i in range(len(vars()['Dfs_'+str(s)])):
        aux=[]
        for n in range(len(vars()['Dfs_'+str(s)][i])):
            if pd.isnull(vars()['Dfs_'+str(s)][i].iloc[n]['AP']) and (pd.isnull(vars()['Dfs_'+str(s)][i].iloc[n]['AS'])) and (pd.isnull(vars()['Dfs_'+str(s)][i].iloc[n]['HST'])) and (vars()['Dfs_'+str(s)][i].iloc[n]['FTAG']==0) and (vars()['Dfs_'+str(s)][i].iloc[n]['FTHG']==0):
                aux+=[vars()['Dfs_'+str(s)][i].index[n]]
        if len(aux)==len(vars()['Dfs_'+str(s)][i]):
            aux=list(vars()['Dfs_'+str(s)][i].sort_values('GameID',ascending=False).index[1:])
        vars()['Drop_'+str(s)]+=aux    
        

In [471]:
for s in range(2012,2021,1):
    vars()['FinalEred_'+str(s)]=vars()['Ered_'+str(s)].drop(vars()['Drop_'+str(s)],axis=0)

In [472]:
for s in range(2012,2021,1):
    print(len(vars()['FinalEred_'+str(s)]))

204
251
229
232
228
228
219
228
178


In [589]:
s=2013
dixH={}
dixA={}
for i in range(len(vars()['FinalEred_'+str(s)])):
    if vars()['FinalEred_'+str(s)].iloc[i]['HT'] not in dixH:
        dixH[vars()['FinalEred_'+str(s)].iloc[i]['HT']]=0
    dixH[vars()['FinalEred_'+str(s)].iloc[i]['HT']]+=1
    if vars()['FinalEred_'+str(s)].iloc[i]['AT'] not in dixA:
        dixA[vars()['FinalEred_'+str(s)].iloc[i]['AT']]=0
    dixA[vars()['FinalEred_'+str(s)].iloc[i]['AT']]+=1
print(len(vars()['FinalEred_'+str(s)]))

228


In [590]:
dixH

{'Celtic': 19,
 'Hearts': 19,
 'Ross County': 19,
 'St Mirren': 19,
 'Kilmarnock': 19,
 'Dundee United': 19,
 'Aberdeen': 19,
 'Inverness CT': 19,
 'Motherwell': 19,
 'Dundee': 19,
 'Hibernian': 19,
 'St. Johnstone': 19}

In [591]:
s=2013
tier1=['Celtic','Hibernian','Kilmarnock','Aberdeen','Rangers','Hearts']
for key in dixH:
    for key2 in dixA:
        if key2==key:
            continue
        else:
            size = len(vars()['FinalEred_'+str(s)].loc[np.logical_or(np.logical_and(vars()['FinalEred_'+str(s)]['HT']==key,vars()['FinalEred_'+str(s)]['AT']==key2),
                                                                     np.logical_and(vars()['FinalEred_'+str(s)]['AT']==key,vars()['FinalEred_'+str(s)]['HT']==key2))])
            if size>4 and (((key in tier1) and (key2 in tier1)) or ((key not in tier1) and (key2 not in tier1))):
                print(key,' Vs ',key2)
                print('Missing')

In [585]:
key='Kilmarnock'
key2='Hibernian'
s=2013
vars()['FinalEred_'+str(s)].loc[np.logical_or(np.logical_and(vars()['FinalEred_'+str(s)]['HT']==key,vars()['FinalEred_'+str(s)]['AT']==key2),
                                                                     np.logical_and(vars()['FinalEred_'+str(s)]['AT']==key,vars()['FinalEred_'+str(s)]['HT']==key2))]

,AAD,ABS,ACCC,ACorn,AF,AOff,AP,APass,AR,AS,ASOT,AST,ASaves,AT,ATack,AY,Attendance,Date,FTAG,FTHG,GameID,HAD,HBS,HCCC,HCorn,HF,HOff,HP,HPass,HR,HS,HSOT,HST,HSaves,HT,HTack,HY,League,Unnamed: 0,Time
43,0.0,0.0,0.0,10.0,12.0,0.0,53.0,0.0,0.0,10.0,8.0,2.0,0.0,Kilmarnock,0.0,2.0,"9,723.",2012-09-15,1.0,2.0,261973.0,0.0,0.0,0.0,3.0,19.0,7.0,47.0,0.0,0.0,10.0,5.0,5.0,0.0,Hibernian,0.0,1.0,Scottish Premiership,NaN,1500
156,0.0,0.0,0.0,6.0,12.0,0.0,43.0,0.0,0.0,10.0,4.0,6.0,0.0,Hibernian,0.0,3.0,"4,028.",2012-12-23,1.0,1.0,279573.0,0.0,0.0,0.0,6.0,14.0,3.0,57.0,0.0,0.0,11.0,3.0,8.0,0.0,Kilmarnock,0.0,0.0,Scottish Premiership,NaN,1245
240,0.0,0.0,0.0,2.0,7.0,5.0,0.0,0.0,0.0,13.0,6.0,7.0,0.0,Kilmarnock,0.0,1.0,"8,121.",2013-02-27,2.0,2.0,262111.0,0.0,0.0,0.0,8.0,13.0,4.0,0.0,0.0,0.0,12.0,8.0,4.0,0.0,Hibernian,0.0,1.0,Scottish Premiership,NaN,1945
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hibernian,NaN,NaN,NaN,2013-05-05,1.0,1.0,285597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kilmarnock,NaN,NaN,Scottish Premiership,NaN,1500
311,0.0,0.0,0.0,5.0,14.0,3.0,53.0,0.0,0.0,8.0,4.0,4.0,0.0,Hibernian,0.0,3.0,"3,198.",2013-05-15,3.0,1.0,286417.0,0.0,0.0,0.0,5.0,8.0,2.0,47.0,0.0,0.0,10.0,6.0,4.0,0.0,Kilmarnock,0.0,0.0,Scottish Premiership,NaN,1945


In [587]:
FinalEred_2013.loc[np.logical_and(FinalEred_2013['AT']=='Hibernian',FinalEred_2013['HT']=='Kilmarnock')]

,AAD,ABS,ACCC,ACorn,AF,AOff,AP,APass,AR,AS,ASOT,AST,ASaves,AT,ATack,AY,Attendance,Date,FTAG,FTHG,GameID,HAD,HBS,HCCC,HCorn,HF,HOff,HP,HPass,HR,HS,HSOT,HST,HSaves,HT,HTack,HY,League,Unnamed: 0,Time
156,0.0,0.0,0.0,6.0,12.0,0.0,43.0,0.0,0.0,10.0,4.0,6.0,0.0,Hibernian,0.0,3.0,"4,028.",2012-12-23,1.0,1.0,279573.0,0.0,0.0,0.0,6.0,14.0,3.0,57.0,0.0,0.0,11.0,3.0,8.0,0.0,Kilmarnock,0.0,0.0,Scottish Premiership,NaN,1245
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hibernian,NaN,NaN,NaN,2013-05-05,1.0,1.0,285597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kilmarnock,NaN,NaN,Scottish Premiership,NaN,1500
311,0.0,0.0,0.0,5.0,14.0,3.0,53.0,0.0,0.0,8.0,4.0,4.0,0.0,Hibernian,0.0,3.0,"3,198.",2013-05-15,3.0,1.0,286417.0,0.0,0.0,0.0,5.0,8.0,2.0,47.0,0.0,0.0,10.0,6.0,4.0,0.0,Kilmarnock,0.0,0.0,Scottish Premiership,NaN,1945


In [415]:
Ered_2020.loc[np.logical_and(Ered_2020['HT']=='Blackburn Rovers',Ered_2020['AT']=='Charlton Athletic')]

,AAD,ABS,ACCC,ACorn,AF,AOff,AP,APass,AR,AS,ASOT,AST,ASaves,AT,ATack,AY,Attendance,Date,FTAG,FTHG,GameID,HAD,HBS,HCCC,HCorn,HF,HOff,HP,HPass,HR,HS,HSOT,HST,HSaves,HT,HTack,HY,League,Unnamed: 0,Time


In [533]:
missing_16 = pd.read_csv('C://Users//joaom//Desktop//WScrapy2//Sky//Missing_Game.csv')
missing_16 = missing_16[:-1]
missing_16

,AAD,ABS,ACCC,ACorn,AF,AOff,AP,APass,AR,AS,ASOT,AST,ASaves,AT,ATack,AY,Attendance,Date,FTAG,FTHG,GameID,HAD,HBS,HCCC,HCorn,HF,HOff,HP,HPass,HR,HS,HSOT,HST,HSaves,HT,HTack,HY,League
0,0.0,1.0,0.0,2.0,6.0,0.0,49.0,0.0,0.0,4.0,2.0,1.0,3.0,Hamilton Academical,0.0,2.0,"4,699.","3:00pm, Saturday 12th May 2018.",0.0,3.0,389003.0,0.0,2.0,0.0,5.0,16.0,0.0,51.0,0.0,0.0,11.0,3.0,6.0,1.0,Motherwell,0.0,0.0,Scottish Premiership


In [534]:
FinalEred_2018=pd.concat([FinalEred_2018,missing_16],sort=False).reset_index(drop=True)

In [588]:
dropz=[304]
FinalEred_2013=FinalEred_2013.drop(dropz,axis=0)

In [304]:
dixA

{'Bolton Wanderers': 23,
 'Wigan Athletic': 24,
 'Watford': 23,
 'Charlton Athletic': 23,
 'Blackpool': 23,
 'Brighton and Hove Albion': 23,
 'Leicester City': 23,
 'Yeovil Town': 23,
 'Huddersfield Town': 23,
 'Sheffield Wednesday': 23,
 'Ipswich Town': 23,
 'Blackburn Rovers': 23,
 'Nottingham Forest': 23,
 'Barnsley': 23,
 'Reading': 23,
 'Derby County': 23,
 'Middlesbrough': 22,
 'Queens Park Rangers': 23,
 'Millwall': 23,
 'Burnley': 23,
 'Bournemouth': 23,
 'Birmingham City': 23,
 'Leeds United': 23,
 'Doncaster Rovers': 24}

In [168]:
missing_12 = pd.read_csv('C://Users//joaom//Desktop//WScrapy2//Sky//Missing_Game.csv')

In [169]:
missing_12=missing_12.head(1)

In [173]:
FinalEred_2012=pd.concat([FinalEred_2012,missing_12],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [188]:
any(pd.isnull(FinalEred_2012.loc[4]))

True

In [284]:
s=2013
vars()['FinalEred_'+str(s)]=vars()['FinalEred_'+str(s)].reset_index(drop=True)
ind=[]
for i in range(len(vars()['FinalEred_'+str(s)])):
    if any(pd.isnull(vars()['FinalEred_'+str(s)].iloc[i])) and (vars()['FinalEred_'+str(s)].iloc[i]['FTHG']==0) and (vars()['FinalEred_'+str(s)].iloc[i]['FTAG']==0):
        #print(vars()['FinalEred_'+str(s)].iloc[i]['FTHG'],vars()['FinalEred_'+str(s)].iloc[i]['FTAG'])
        #print(vars()['FinalEred_'+str(s)].iloc[i]['HT'],vars()['FinalEred_'+str(s)].iloc[i]['AT'])
        #print(i)
        ind+=[i]
vars()['FinalEred_'+str(s)].loc[ind]

,AAD,ABS,ACCC,ACorn,AF,AOff,AP,APass,AR,AS,ASOT,AST,ASaves,AT,ATack,AY,Attendance,Date,FTAG,FTHG,GameID,HAD,HBS,HCCC,HCorn,HF,HOff,HP,HPass,HR,HS,HSOT,HST,HSaves,HT,HTack,HY,League,Unnamed: 0,Time
15,0.0,0.0,0.0,5.0,10.0,2.0,45.0,0.0,0.0,10.0,4.0,6.0,0.0,Cardiff City,0.0,1.0,"25,518.",2012-08-21,0.0,0.0,262386.0,0.0,0.0,0.0,14.0,5.0,3.0,55.0,0.0,0.0,16.0,7.0,9.0,0.0,Brighton and Hove Albion,0.0,0.0,Sky Bet Championship,NaN,1945
30,0.0,0.0,0.0,6.0,9.0,2.0,51.0,0.0,0.0,13.0,5.0,8.0,0.0,Hull City,0.0,0.0,"16,202.",2012-08-25,0.0,0.0,262482.0,0.0,0.0,0.0,8.0,10.0,1.0,49.0,0.0,0.0,9.0,3.0,6.0,0.0,Charlton Athletic,0.0,0.0,Sky Bet Championship,NaN,1500
107,0.0,0.0,0.0,5.0,5.0,0.0,44.0,0.0,0.0,5.0,4.0,1.0,0.0,Blackburn Rovers,0.0,0.0,"18,748.",2012-10-03,0.0,0.0,267257.0,0.0,0.0,0.0,9.0,8.0,0.0,56.0,0.0,0.0,17.0,16.0,1.0,0.0,Nottingham Forest,0.0,0.0,Sky Bet Championship,NaN,1945
114,0.0,0.0,0.0,7.0,11.0,0.0,54.0,0.0,0.0,11.0,8.0,3.0,0.0,Brighton and Hove Albion,0.0,2.0,"22,059.",2012-10-06,0.0,0.0,262392.0,0.0,0.0,0.0,11.0,13.0,0.0,46.0,0.0,0.0,4.0,2.0,2.0,0.0,Derby County,0.0,1.0,Sky Bet Championship,NaN,1500
176,0.0,0.0,0.0,2.0,10.0,4.0,47.0,0.0,0.0,13.0,8.0,5.0,0.0,Millwall,0.0,2.0,"11,293.",2012-11-06,0.0,0.0,262653.0,0.0,0.0,0.0,4.0,11.0,1.0,53.0,0.0,0.0,15.0,9.0,6.0,0.0,Watford,0.0,3.0,Sky Bet Championship,NaN,1945
177,0.0,0.0,0.0,5.0,5.0,4.0,49.0,0.0,0.0,9.0,7.0,2.0,0.0,Middlesbrough,0.0,1.0,"20,150.",2012-11-06,0.0,0.0,262662.0,0.0,0.0,0.0,7.0,7.0,1.0,51.0,0.0,0.0,13.0,9.0,4.0,0.0,Nottingham Forest,0.0,1.0,Sky Bet Championship,NaN,1945
178,0.0,0.0,0.0,16.0,13.0,1.0,43.0,0.0,0.0,15.0,5.0,10.0,0.0,Leicester City,0.0,0.0,"16,754.",2012-11-06,0.0,0.0,262324.0,0.0,0.0,0.0,7.0,10.0,2.0,57.0,0.0,0.0,10.0,1.0,9.0,0.0,Bolton Wanderers,0.0,2.0,Sky Bet Championship,NaN,2000
220,0.0,0.0,0.0,3.0,9.0,2.0,55.0,0.0,0.0,4.0,1.0,3.0,0.0,Crystal Palace,0.0,1.0,"16,656.",2012-11-27,0.0,0.0,262520.0,0.0,0.0,0.0,12.0,11.0,4.0,45.0,0.0,0.0,17.0,11.0,6.0,0.0,Hull City,0.0,4.0,Sky Bet Championship,NaN,1945
235,0.0,0.0,0.0,5.0,18.0,2.0,48.0,0.0,0.0,3.0,1.0,2.0,0.0,Charlton Athletic,0.0,1.0,"18,013.",2012-12-01,0.0,0.0,262493.0,0.0,0.0,0.0,8.0,7.0,2.0,52.0,0.0,0.0,15.0,9.0,6.0,0.0,Millwall,0.0,0.0,Sky Bet Championship,NaN,1500
257,0.0,0.0,0.0,3.0,18.0,0.0,29.0,0.0,0.0,2.0,2.0,0.0,0.0,Nottingham Forest,0.0,2.0,"26,684.",2012-12-15,0.0,0.0,262401.0,0.0,0.0,0.0,6.0,7.0,0.0,71.0,0.0,0.0,7.0,7.0,0.0,0.0,Brighton and Hove Albion,0.0,1.0,Sky Bet Championship,NaN,1500


In [37]:
file.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True).to_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//SkyScott___//SkyScott17_18.xlsx',index=False)

In [29]:
file = pd.read_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//SkyScott___//SkyScott17_18.xlsx')

In [36]:
len(list(map(lambda x:type(x),list(file['Date']))))

228

In [31]:
Ered1 = file.tail(9)

In [32]:
dtime = list(map(lambda x:SkySports_Date(x),list(Ered1['Date'])))
time = [i[1] for i in dtime]
date = [i[0] for i in dtime]
Ered1['Date'] = date
Ered1['Time'] = time

C:\Users\joaom\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\joaom\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
Ered1

,Unnamed: 0,Unnamed: 0.1,AAD,ABS,ACCC,ACorn,AF,AOff,AP,APass,AR,AS,ASOT,AST,ASaves,AT,ATack,AY,Attendance,Date,FTAG,FTHG,GameID,HAD,HBS,HCCC,HCorn,HF,HOff,HP,HPass,HR,HS,HSOT,HST,HSaves,HT,HTack,HY,League,Time,FTR
219,219,219,0,2,0,4,22,0,41,0,0,8,4,2,8,Rangers,0,4,"58,320.",2018-04-29,0,5,388989,0,7,0,8,6,0,59,0,0,24,4,13,2,Celtic,0,1,Scottish Premiership,1200,H
220,220,220,0,3,0,3,12,0,56,0,0,13,6,4,3,Celtic,0,2,"19,886.",2018-04-21,1,2,388979,0,1,0,6,12,0,44,0,0,13,7,5,3,Hibernian,0,2,Scottish Premiership,1230,H
221,221,221,0,1,0,2,6,0,49,0,0,4,2,1,3,Hamilton Academical,0,2,"4,699.",2018-05-12,0,3,389003,0,2,0,5,16,0,51,0,0,11,3,6,1,Motherwell,0,0,Scottish Premiership,1500,H
222,222,222,0,1,0,3,7,0,52,0,0,10,4,5,1,Aberdeen,0,1,"5,067.",2018-04-21,2,0,388980,0,2,0,8,10,0,48,0,0,9,6,1,3,Kilmarnock,0,1,Scottish Premiership,1500,A
223,223,223,0,1,0,1,14,0,37,0,0,8,4,3,3,Hamilton Academical,0,3,"3,839.",2018-04-21,1,2,388981,0,4,0,5,8,0,63,0,0,11,2,5,2,Partick Thistle,0,2,Scottish Premiership,1500,H
224,224,224,0,1,0,8,10,0,52,0,0,8,5,2,2,Motherwell,0,1,"3,879.",2018-04-21,0,0,388982,0,2,0,4,15,0,48,0,0,8,4,2,2,Ross County,0,3,Scottish Premiership,1500,D
225,225,225,0,2,0,2,11,0,52,0,0,10,5,3,5,Partick Thistle,0,3,"2,925.",2018-04-28,1,1,388988,0,0,0,4,20,0,48,0,0,8,2,6,2,St. Johnstone,0,3,Scottish Premiership,1500,D
226,226,226,0,0,0,2,12,0,46,0,0,5,0,5,4,Kilmarnock,0,3,"17,470.",2018-04-28,3,5,388986,0,3,0,5,13,0,54,0,0,15,3,9,2,Hibernian,0,2,Scottish Premiership,1500,H
227,227,227,0,3,0,3,11,0,48,0,0,13,6,4,2,Hibernian,0,2,"19,324.",2018-05-09,1,2,389001,0,1,0,5,15,0,52,0,0,6,1,4,5,Hearts,0,1,Scottish Premiership,1945,H


In [33]:
file = file[:-9]

In [34]:
file = pd.concat([file,Ered1],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [15]:
file.head(20)

,Unnamed: 0,Unnamed: 0.1,AAD,ABS,ACCC,ACorn,AF,AOff,AP,APass,AR,AS,ASOT,AST,ASaves,AT,ATack,AY,Attendance,Date,FTAG,FTHG,GameID,HAD,HBS,HCCC,HCorn,HF,HOff,HP,HPass,HR,HS,HSOT,HST,HSaves,HT,HTack,HY,League,Time,FTR
0,551,551,47.5,4,1,11,20,2,50.1,74.7,0,17,5,8,1,Middlesbrough,75.0,2,"10,053.",2019-08-02 00:00:00,3,3,409348,52.5,5,2,8,14,2,49.9,72.9,0,12,3,4,5,Luton Town,91.7,2,Sky Bet Championship,1945.0,D
1,542,542,63.6,3,1,5,14,1,54.8,75.9,0,15,8,4,2,Fulham,70.6,1,"14,823.",2019-08-03 00:00:00,0,1,409349,36.4,4,3,7,16,2,45.2,76.1,0,17,9,4,3,Barnsley,50.0,3,Sky Bet Championship,1500.0,H
2,543,543,28.2,2,2,2,13,2,38.0,64.2,0,9,2,5,1,Charlton Athletic,78.6,3,"14,184.",2019-08-03 00:00:00,2,1,409350,71.8,6,1,12,15,5,62.0,78.6,0,14,6,2,3,Blackburn Rovers,44.4,0,Sky Bet Championship,1500.0,A
3,544,544,53.8,0,0,2,12,2,24.2,55.1,0,1,0,1,3,Birmingham City,57.1,3,"11,332.",2019-08-03 00:00:00,1,0,409351,46.2,3,2,6,10,0,75.8,85.7,0,15,9,3,0,Brentford,100.0,1,Sky Bet Championship,1500.0,A
4,545,545,34.7,1,1,7,18,2,62.8,76.3,0,6,4,1,2,Preston North End,100.0,3,"14,923.",2019-08-03 00:00:00,0,1,409352,65.3,4,0,8,11,1,37.2,53.5,0,11,4,3,0,Millwall,71.4,2,Sky Bet Championship,1500.0,H
5,546,546,47.2,2,2,3,15,0,40.6,76.7,1,13,6,5,2,Sheffield Wednesday,64.3,3,"16,319.",2019-08-03 00:00:00,3,1,409354,52.8,4,1,7,12,1,59.4,80.0,0,14,6,4,2,Reading,100.0,0,Sky Bet Championship,1500.0,A
6,547,547,47.7,1,2,0,14,3,44.9,71.6,0,6,2,3,0,Queens Park Rangers,81.8,3,"24,004.",2019-08-03 00:00:00,2,1,409355,52.3,1,1,7,18,0,55.1,73.4,0,12,10,1,1,Stoke City,76.2,1,Sky Bet Championship,1500.0,A
7,548,548,50.0,4,0,4,10,1,36.9,69.7,0,12,4,4,6,Hull City,42.3,2,"15,741.",2019-08-03 00:00:00,1,2,409356,50.0,5,1,10,9,2,63.1,83.1,0,19,6,8,3,Swansea City,77.8,3,Sky Bet Championship,1500.0,H
8,549,549,62.5,4,1,6,15,2,43.6,61.0,0,13,5,4,1,Cardiff City,53.8,4,"12,169.",2019-08-03 00:00:00,2,3,409357,37.5,5,1,6,9,0,56.4,68.4,0,12,3,4,2,Wigan Athletic,81.8,2,Sky Bet Championship,1500.0,H
9,550,550,61.5,4,0,5,6,0,54.8,79.6,0,12,5,3,3,West Bromwich Albion,66.7,1,"27,592.",2019-08-03 00:00:00,2,1,409353,38.5,4,0,9,8,2,45.2,78.6,0,15,7,4,1,Nottingham Forest,60.9,0,Sky Bet Championship,1730.0,A
